In [1]:
import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics 8.3.40  Python-3.10.6 torch-2.4.1+cu124 CUDA:0 (NVIDIA GeForce RTX 4060, 8187MiB)
Setup complete  (12 CPUs, 15.9 GB RAM, 1101.2/1863.0 GB disk)


In [8]:
def center_crop(frame, target_size=(640, 640)):
    h, w, _ = frame.shape
    crop_h, crop_w = target_size

    # Calculate cropping coordinates
    start_x = max(0, (w - crop_w) // 2)
    start_y = max(0, (h - crop_h) // 2)
    end_x = start_x + crop_w
    end_y = start_y + crop_h

    # Perform cropping
    cropped_frame = frame[start_y:end_y, start_x:end_x]
    return cropped_frame

# Object Detection

In [2]:
model = YOLO("last_seg_weight.pt")

In [3]:
model.export(format="openvino", int8=True, data="datasets/dataset640\data.yaml")

Ultralytics 8.3.40  Python-3.10.6 torch-2.4.1+cu124 CPU (Intel Core(TM) i7-8700K 3.70GHz)
YOLO11s-seg summary (fused): 265 layers, 10,067,203 parameters, 0 gradients, 35.3 GFLOPs

PyTorch: starting from 'last_seg_weight.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) ((1, 37, 8400), (1, 32, 160, 160)) (19.6 MB)

OpenVINO: starting export with openvino 2024.5.0-17288-7975fa5da0c-refs/pull/3856/head...
INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, openvino
OpenVINO: collecting INT8 calibration images from 'data=datasets/dataset640\data.yaml'


Scanning E:\universita\aml\final_pokemon_project\datasets\dataset640\labels\val... 1000 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1000/1000 [00:01<00:00, 524.38it/s]


New cache created: E:\universita\aml\final_pokemon_project\datasets\dataset640\labels\val.cache
INFO:nncf:16 ignored nodes were found by patterns in the NNCFGraph
INFO:nncf:1 ignored nodes were found by types in the NNCFGraph
INFO:nncf:Not adding activation input quantizer for operation: 194 __module.model.23.dfl/aten::view/Reshape
INFO:nncf:Not adding activation input quantizer for operation: 195 __module.model.23/aten::sigmoid/Sigmoid
INFO:nncf:Not adding activation input quantizer for operation: 208 __module.model.23.dfl/aten::transpose/Transpose
INFO:nncf:Not adding activation input quantizer for operation: 220 __module.model.23.dfl/aten::softmax/Softmax
INFO:nncf:Not adding activation input quantizer for operation: 230 __module.model.23.dfl.conv/aten::_convolution/Convolution
INFO:nncf:Not adding activation input quantizer for operation: 240 __module.model.23.dfl/aten::view/Reshape_1
INFO:nncf:Not adding activation input quantizer for operation: 262 __module.model.23/aten::sub/Sub

c:\Users\baeda\AppData\Local\Programs\Python\Python310\lib\site-packages\nncf\tensor\tensor.py:100: RuntimeWarning: invalid value encountered in multiply
  return Tensor(self.data * unwrap_tensor_data(other))


OpenVINO: export success  145.3s, saved as 'last_seg_weight_int8_openvino_model\' (10.5 MB)

Export complete (146.1s)
Results saved to E:\universita\aml\final_pokemon_project
Predict:         yolo predict task=segment model=last_seg_weight_int8_openvino_model imgsz=640 int8 
Validate:        yolo val task=segment model=last_seg_weight_int8_openvino_model imgsz=640 data=yolov11.yaml int8 
Visualize:       https://netron.app


'last_seg_weight_int8_openvino_model'

## Webcam connected on pc

In [11]:
import cv2

# Initialize the webcam (use 0 for default webcam, or 1, 2, etc., for external webcams)
cap = cv2.VideoCapture('binder.mp4')

# model = YOLO("runs/detect/train5/weights/best.pt")

# Check if the webcam is opened correctly
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# Loop to capture frames
while True:
    # Capture frame-by-frame
    ret, frame = cap.read()
    
    # If frame is read correctly, ret will be True
    if not ret:
        print("Error: Cannot read frame.")
        break

    # results = model.predict(frame, imgsz=480)
    cropped_frame = center_crop(frame, target_size=(844, 844))
    results = model.predict(cropped_frame, imgsz=844)
    for result in results:
        for box in result.boxes:
            # Get bounding box coordinates
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Convert to integers
            conf = box.conf[0]  # Confidence score
            cls = int(box.cls[0])  # Class index
            label = model.names[cls]  # Class label from model

            # Draw the bounding box
            cv2.rectangle(cropped_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            # Add label and confidence score
            cv2.putText(cropped_frame, f"{label} {conf:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    # Display the resulting frame
    cv2.imshow('Webcam', cropped_frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()



WARNING  imgsz=[844] must be multiple of max stride 32, updating to [864]
0: 512x864 (no detections), 109.0ms
Speed: 7.0ms preprocess, 109.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 864)

WARNING  imgsz=[844] must be multiple of max stride 32, updating to [864]
0: 512x864 (no detections), 16.0ms
Speed: 5.0ms preprocess, 16.0ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 864)

WARNING  imgsz=[844] must be multiple of max stride 32, updating to [864]
0: 512x864 (no detections), 13.7ms
Speed: 4.0ms preprocess, 13.7ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 864)

WARNING  imgsz=[844] must be multiple of max stride 32, updating to [864]
0: 512x864 (no detections), 18.0ms
Speed: 3.0ms preprocess, 18.0ms inference, 0.0ms postprocess per image at shape (1, 3, 512, 864)

WARNING  imgsz=[844] must be multiple of max stride 32, updating to [864]
0: 512x864 (no detections), 17.9ms
Speed: 5.1ms preprocess, 17.9ms inference, 0.0ms postprocess

## Phone camera

Steps:
1. Install **IP Webcam** (Thyoni Tech)
2. Scroll down and click on **Start Server** (make sure that PC and phone are connected on network)
3. Change the url

In [56]:
# Import essential libraries 
import requests 
import cv2 
import numpy as np 

# Replace the below URL with your own. Make sure to add "/shot.jpg" at last. 
url = "http://192.168.188.30:8080/shot.jpg"

# While loop to continuously fetching data from the Url 
while True: 
	img_resp = requests.get(url) 
	img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8) 
	frame = cv2.imdecode(img_arr, -1) 
	# cropped_frame = center_crop(frame, target_size=(480, 480))
	cropped_frame = frame
	results = model.predict(cropped_frame, imgsz=480)
	for result in results:
		for box in result.boxes:
			# Get bounding box coordinates
			x1, y1, x2, y2 = map(int, box.xyxy[0])  # Convert to integers
			conf = box.conf[0]  # Confidence score
			cls = int(box.cls[0])  # Class index
			label = model.names[cls]  # Class label from model

			# Draw the bounding box
			cv2.rectangle(cropped_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
			# Add label and confidence score
			cv2.putText(cropped_frame, f"{label} {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

	cv2.imshow("Android_cam", cropped_frame) 

	# Press Esc key to exit 
	if cv2.waitKey(1) & 0xFF == ord('q'):
		break

cv2.destroyAllWindows() 



0: 288x480 (no detections), 11.0ms
Speed: 1.0ms preprocess, 11.0ms inference, 1.0ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 13.0ms
Speed: 1.0ms preprocess, 13.0ms inference, 0.5ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 12.0ms
Speed: 1.0ms preprocess, 12.0ms inference, 0.0ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 24.0ms
Speed: 2.0ms preprocess, 24.0ms inference, 1.0ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 25.0ms
Speed: 1.0ms preprocess, 25.0ms inference, 2.0ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 15.0ms
Speed: 1.0ms preprocess, 15.0ms inference, 1.0ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 (no detections), 24.0ms
Speed: 1.0ms preprocess, 24.0ms i

# Segmentation

In [6]:
def reorder_points(points):
    # Calculate centroids
    centroids = np.mean(points, axis=0)

    # Sort points based on distance from centroids
    points_sorted = sorted(points, key=lambda x: np.arctan2(x[0][1] - centroids[0][1], x[0][0] - centroids[0][0]))

    return np.array(points_sorted)

def get_card_dimensions(corners):
    # Calculate the width and height of the card
    width = np.linalg.norm(corners[0] - corners[1])
    height = np.linalg.norm(corners[1] - corners[2])
    return width, height

def perspective_transform(image, mask, mirror):
    # Convert the boolean mask to uint8
    mask_uint8 = mask.astype(np.uint8) * 255

    # Visualize the mask
    h, w = mask_uint8.shape
    mask_visualized = np.zeros((h, w, 3), dtype=np.uint8)
    # Set the background pixels to black
    mask_visualized[:, :] = [0, 0, 0]
    # Set the object mask pixels to red
    mask_visualized[mask_uint8 != 0] = [0, 0, 255]
    # viewer.VideoFrameBuilder().add_image(mask_visualized, 2, "Segmentation Mask")

    # Find contours in the mask
    contours, _ = cv2.findContours(mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    image_with_detections = image.copy()

    # Approximate the contours to get the four corners of the card
    for contour in contours:
        epsilon = 0.1 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        if len(approx) == 4:
            # Rearrange the points in the approx array if needed
            approx = reorder_points(approx)

            # Draw the contour on the image
            cv2.drawContours(image_with_detections, [approx], -1, (0, 0, 255), 2)

            # Draw the corners on the image
            circle_size = 8
            for point in approx:
                x, y = point.ravel()
                cv2.circle(image_with_detections, (x, y), circle_size, (0, 255, 0), -1)

            # viewer.VideoFrameBuilder().add_image(image_with_detections, 2, "Edge & Corner Detection")

            # Perform perspective transform
            width, height = get_card_dimensions(approx)
            dst = np.array([[0, 0], [width - 1, 0], [width - 1, height - 1], [0, height - 1]], dtype=np.float32)
            M = cv2.getPerspectiveTransform(approx.astype(np.float32), dst)

            warped = cv2.warpPerspective(image, M, (int(width), int(height)))

            # Rotate the warped image to ensure portrait orientation
            if width > height:
                warped = cv2.rotate(warped, cv2.ROTATE_90_CLOCKWISE)

            # Resize the warped image to fill the canvas without maintaining aspect ratio
            warped_stretched = cv2.resize(warped, (320, 320))

            if mirror is True:
                warped_stretched = cv2.flip(warped_stretched, 1)

            return warped_stretched

    return None


import imagehash
hash_size = 16 # bytes
def hash_image(img):
    # Convert the NumPy array to a PIL image
    img = Image.fromarray(img)

    img = img.convert('RGB')

    # Resize the image to the desired size
    #img = img.resize((image_size, image_size))

    # Compute the hash
    # ahash = str(imagehash.average_hash(img, hash_size))
    dhash = imagehash.dhash(img, hash_size)
    phash = imagehash.phash(img, hash_size)

    hash = f'{dhash}{phash}'

    return hash

import json
from PIL import Image
hash_filename = 'hashes_dphash_16.json'
min_similarity = 14*6.8 # lower is more exact
check_flipped = False # if no match, rotate 180 and check again

def find_flipped_match(card_image):
    card_image = cv2.rotate(card_image, cv2.ROTATE_180)
    image_hash = hash_image(card_image)
    match, _ = find_match(image_hash)
    return match

def hamming_distance(hash1, hash2):
    assert len(hash1) == len(hash2), "Hash lengths are not equal"
    return sum(ch1 != ch2 for ch1, ch2 in zip(hash1, hash2))

def find_match(hash_a):
    best_match = None
    min_sim = min_similarity

    for card_id, data in hash_dict.items():
        hash_b = data['hash']
        similarity = hamming_distance(hash_a, hash_b)
        if similarity < min_sim:
            min_sim = similarity
            best_match = card_id

    if best_match is None:
        return None, None

    matchString = f"{hash_dict[best_match]['name']} {hash_dict[best_match]['id']}"

    return matchString, min_sim


with open(f'{hash_filename}', 'r', encoding='utf-8') as json_file:
    hash_dict = json.load(json_file)

def get_match_pool(image, mirror):
    # if 'match' in detection:
    #     return detection['match']
    # if 'track_id' not in detection:
    #     return None
    # card_image = perspective_transform(image, detection['mask'], mirror)
    card_image = image
    if card_image is None:
        return None
    image_hash = hash_image(card_image)
    match, _ = find_match(image_hash)
    if match is None and check_flipped is True:
        match = find_flipped_match(card_image)
    if match is not None:
        return match
    return None

In [42]:
model = YOLO("runs/segment/train5/weights/best.pt")

In [13]:
# Import essential libraries  
import requests 
import cv2 
import numpy as np 
from sort import Sort  # Import SORT tracker

# Replace the below URL with your own. Make sure to add "/shot.jpg" at last. 
url = "http://192.168.188.30:8080/shot.jpg"

alpha = 0.5

extracted_cards = []

# Initialize the SORT tracker
tracker = Sort()

# While loop to continuously fetching data from the URL
while True:
    # Fetch the frame from the Android camera
    img_resp = requests.get(url)
    img_arr = np.array(bytearray(img_resp.content), dtype=np.uint8)
    frame = cv2.imdecode(img_arr, -1)

    # Run YOLO prediction on the frame
    results = model(frame)

    # Get the masks and bounding boxes from the results
    if results[0].masks:  # Check if masks exist
        masks = results[0].masks.data.cpu().numpy()  # Convert masks to NumPy array
        boxes = results[0].boxes.xyxy.cpu().numpy()  # Bounding boxes (x_min, y_min, x_max, y_max)
        confidences = results[0].boxes.conf.cpu().numpy()  # Confidence scores
        classes = results[0].boxes.cls.cpu().numpy()  # Class IDs

        # Prepare detections for SORT tracker
        detections = np.hstack((boxes, confidences[:, np.newaxis]))  # [x_min, y_min, x_max, y_max, confidence]

        # Update tracker with current frame's detections
        tracked_objects = tracker.update(detections)

        # Overlay transparent red masks and track objects
        for i, tracked_obj in enumerate(tracked_objects):
            x_min, y_min, x_max, y_max, track_id = map(int, tracked_obj)

            # Find corresponding mask
            corresponding_index = None
            for j, box in enumerate(boxes):
                if np.allclose(box, [x_min, y_min, x_max, y_max], atol=5):
                    corresponding_index = j
                    break

            if corresponding_index is not None:
                mask = masks[corresponding_index]
                mask_resized = cv2.resize(mask, (frame.shape[1], frame.shape[0]))  # Resize mask if needed
                mask_binary = (mask_resized > 0.5).astype(np.uint8)  # Threshold the mask

                # Transform and match card only if it hasn't been processed
                if track_id not in extracted_cards:
                    transformed_img = perspective_transform(frame, mask_resized, False)
                    card_id = get_match_pool(transformed_img, False)  # Get card_id from transformed image
                    extracted_cards.append(track_id)  # Mark as processed

                    # Create a red mask
                    red_mask = np.zeros_like(frame, dtype=np.uint8)
                    red_mask[:, :, 2] = mask_binary * 255  # Set red channel to maximum

                    # Apply transparency by blending the red mask with the original frame
                    frame = np.where(
                        mask_binary[:, :, None] == 1,
                        cv2.addWeighted(frame, 1 - alpha, red_mask, alpha, 0),
                        frame
                    )

                    # Display card_id above the mask
                    cv2.putText(
                        frame, f"Card ID: {card_id}", (x_min, y_min - 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2
                    )

            # Draw bounding box and track ID
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            cv2.putText(frame, f"Track ID: {track_id}", (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the frame with the transparent red mask and bounding boxes
    cv2.imshow("Android_cam", frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()




0: 384x640 (no detections), 119.0ms
Speed: 4.0ms preprocess, 119.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 50.0ms
Speed: 6.0ms preprocess, 50.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.0ms
Speed: 2.0ms preprocess, 24.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 24.0ms
Speed: 2.0ms preprocess, 24.0ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.0ms
Speed: 3.0ms preprocess, 21.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 22.0ms
Speed: 2.0ms preprocess, 22.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 22.0ms
Speed: 2.0ms preprocess, 22.0ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 21.0ms
Speed: 2.0ms preprocess, 21.0ms